In [1]:
%reset -f
import tensorflow as tf
tf.reset_default_graph() 
import matplotlib.pylab as plt
import numpy as np
import pdb
%matplotlib inline

Now we are going to use the CIPHAR10 dataset from https://www.cs.toronto.edu/~kriz/cifar.html. Helpfully, the data is available in picked format which we can read with the following function.

Let's take a look at the first image. Originally this is a 3x1024 vector, representing all the colour channels. We want to display this with matplotlib, so we need to convert is to a 32x32x3 numpy array. In the mnist example, we were able to very easily extract batch of labels and images. We need to write our own way to extract batches of images. Since each chunk has size 10000, I'm going to extract out 100 batches per chunk, at a batch size of 100. We implement these in the following classes.

In [2]:
class Chunk():
    def __init__(self,chunk_filename):
        self.chunk = self.unpickle(chunk_filename)
        self.images = np.array([self.format_image(image) for image in self.chunk['data']])
        self.labels = np.array([self.one_hot(label) for label in self.chunk['labels']])
        self.idx = 0
        
    def one_hot(self,key):
        vector = np.zeros(10, dtype=int)
        vector[key] = int(1)
        return vector
                       
        
    def unpickle(self,chunk):
        import cPickle
        fo = open(chunk, 'rb')
        dictionary = cPickle.load(fo)
        fo.close()
        return dictionary
    def format_image(self,vector):
        return vector.reshape(-1,32,32).transpose(1,2,0)
    
    def next_batch(self,number):
        if self.idx < len(self.labels):
            batch_images = self.images[self.idx : self.idx + number]
            batch_labels = self.labels[self.idx : self.idx + number]
            self.idx += number
            return batch_images, batch_labels
        else:
            raise Exception('Reached end of chunk')
    
        

class CIPHAR10():
    def __init__(self,test_filepath,train_filepaths):
        self.test_data = self.load_data(test_filepath)
        self.train_data = [self.load_data(chunk) for chunk in train_filepaths]
        
    def load_data(self,filepath):
        return Chunk(filepath)

train_filepaths = ['./data/cifar-10-batches-py/data_batch_{}'.format(i) for i in range(1,6)]
test_filepaths = './data/cifar-10-batches-py/test_batch'

We update our X placeholder to take in these 32x32x3 arrays. Y remains unchanged as there are still 10 classes.

In [3]:
def variable(shape):
    if type(shape) is not list:
        shape = [shape]
    return tf.Variable(tf.random_normal(shape))

def conv2dlayer(x, filt, s=2):
    b = filt[-1]
    filt = variable(filt)
    x = tf.nn.conv2d(x, filt, strides=[1,s,s,1], padding='SAME')
    x = tf.nn.bias_add(x,variable([b]))
    return tf.nn.relu(x)

def max2dpool(x, k=2, s=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,s,s,1], padding='SAME')

def full_conn(x, hidden_size):
    flat = np.prod(x.get_shape().as_list()[1:])
    fc = tf.reshape(x, [-1, flat] )
    fc = tf.matmul(fc, variable([flat,hidden_size]))
    fc = tf.nn.bias_add(fc, variable([hidden_size]))
    return tf.nn.relu(fc)
    
def pred_layer(x, n_classes):
    y = tf.matmul(x, variable([x.get_shape().as_list()[-1],10]))
    y = tf.nn.bias_add(y, variable(n_classes))
    return y

def lrn(x):
    return tf.nn.lrn(x, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

In [4]:
X = tf.placeholder(tf.float32, shape = [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, shape = [None, 10])

conv1 = conv2dlayer(X, [5,5,3,32],s=1)
pool1 = max2dpool(conv1, k=3, s=2)
# lrn1 = lrn(pool1)
conv2 = conv2dlayer(pool1, [5,5,32,64], s=1)
pool2 = max2dpool(conv2, k=3, s=2)
# lrn2 = lrn(pool2)
fc1 = full_conn(pool2, 300)
y = pred_layer(fc1, 10)

In [7]:
cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y,Y))
optimiser = tf.train.AdamOptimizer(0.0001).minimize(cost)
results = tf.cast(tf.equal(tf.arg_max(y,1), tf.arg_max(Y,1)), tf.float32)
count = tf.reduce_sum(results)
accuracy = tf.reduce_mean(results)
init = tf.initialize_all_variables()

batch_size = 10
n_batches = 1000
epochs = 10
n_random_select = 200

In [8]:

with tf.Session() as sess:
    sess.run(init)
    ciphar10 = CIPHAR10(test_filepaths,train_filepaths)
    for ep in range(epochs):
        for chunk in ciphar10.train_data:
            for b in range(n_batches):
                b_images, b_labels = chunk.next_batch(batch_size)
                _, c = sess.run([optimiser, cost], feed_dict={X:b_images, Y:b_labels})
            chunk.idx = 0
        
#         if ep % 10 == 0:
        total_count = 0
        for i in range(1000):
            t_batch = ciphar10.test_data.images[10*i:10*i + 10]
            t_labels = ciphar10.test_data.labels[10*i:10*i + 10]
            cnt, acc = sess.run([count,accuracy], feed_dict={X: t_batch, Y: t_labels})
            total_count += cnt
        print "Accuracy on total test set: {}".format(total_count/100)
            

#         if ep % 10 == 0:


Accuracy on total test set: 26.77
Accuracy on total test set: 30.94
Accuracy on total test set: 33.89
Accuracy on total test set: 35.23
Accuracy on total test set: 36.65
Accuracy on total test set: 38.13
Accuracy on total test set: 39.39
Accuracy on total test set: 40.12
Accuracy on total test set: 40.9
Accuracy on total test set: 41.4


In [ ]:
sess.close()

We see that we only reach a maximum of around 40%. This is certainly a harder problem!